In [1]:
import sys
from pathlib import Path
sys.path.append(Path("../modules").absolute().__str__())

from nltk.translate.bleu_score import corpus_bleu
import argparse
import json
import logging
import nltk
import os
import pandas as pd
from evaluation_utils import compute_precision, compute_recall, corpus_meteor, average_precision, is_correct_SPARQL_query, find_id_column, cross_product_func
from data_utils import eval_dataset, get_nested_values, load_dataset, safe_loc, series_or_dataframe_to_list, make_dataframe_from_sparql_response

In [2]:
arguments = {
    "dataset": "../datasets/Mistral-7B-Instruct-v0.2_rv16-ld0.05-bs1-p0-nta1-e3-basic_engpeft-t0.2-topp0.95_executed.parquet.gzip",
    "preprocess_gold": "../datasets/preprocessed_gold.json",
    "model": "Mistral-7B-Instruct-v0.2",
    "output": ".",
    "save_name": "test",
    "log_level": "warning",
    "log_file": "",
}

In [3]:
args = argparse.Namespace()
args.__dict__.update(arguments)
print(args)

numeric_log_level = getattr(logging, args.log_level.upper(), None)
if not isinstance(numeric_log_level, int):
    raise ValueError(f"Invalid log level: {args.log_level}.")
logging.basicConfig(filename=args.log_file if args.log_file else None, level=numeric_log_level)

if not os.path.exists(args.dataset):
    raise FileNotFoundError(f"The dataset file not found with path: {args.dataset}")

if args.preprocess_gold != None and not os.path.exists(args.preprocess_gold):
    raise FileNotFoundError(f"The preprocess gold dataset file not found with path: {args.preprocess_gold}")

nltk.download('wordnet', quiet=True)

Namespace(dataset='../datasets/Mistral-7B-Instruct-v0.2_rv16-ld0.05-bs1-p0-nta1-e3-basic_engpeft-t0.2-topp0.95_executed.parquet.gzip', preprocess_gold='../datasets/preprocessed_gold.json', model='Mistral-7B-Instruct-v0.2', output='.', save_name='test', log_level='warning', log_file='')


True

In [4]:
df = load_dataset(args.dataset)
df_no_gen_fail = df # df.drop(failed_generation_index(df))
df_exec_timeout = df_no_gen_fail.loc[df_no_gen_fail['execution'] == 'timeout']
df_exec_fail = df_no_gen_fail.loc[df_no_gen_fail['execution'].str.startswith('exception')]
df_exec_empty = df_no_gen_fail.loc[df_no_gen_fail['execution'].isnull()]
df_exec_to_eval = df_no_gen_fail.drop(df_exec_timeout.index).drop(df_exec_fail.index).drop(df_exec_empty.index)
df_eval = eval_dataset(df_exec_to_eval)
df_eval['get_nested_values'] = df_eval.apply(lambda x: get_nested_values(x['eval']), axis=1)
df_eval['eval_df'] = df_eval.apply(lambda x: make_dataframe_from_sparql_response(x['eval']), axis=1)
df_eval['id_column'] = df_eval.apply(lambda x: x['eval_df'][find_id_column(x['eval_df'])] if find_id_column(x['eval_df']) != None else None, axis=1)

In [5]:
df_gold_eval = None
# if args.gold != None:
#     df_gold = load_dataset(args.gold)
#     df_gold_exec_timeout = df_gold.loc[df_gold['execution'] == 'timeout']
#     df_gold_exec_fail = df_gold.loc[df_gold['execution'].str.startswith('exception')]
#     df_gold_exec_empty = df_gold.loc[df_gold['execution'].isnull()]
#     df_gold_exec_to_eval = df_gold.drop(df_gold_exec_timeout.index).drop(df_gold_exec_fail.index).drop(df_gold_exec_empty.index)
#     df_gold_eval = eval_dataset(df_gold_exec_to_eval, "gold_eval")
#     df_gold_eval['gold_get_nested_values'] = df_gold_eval.apply(lambda x: get_nested_values(x['gold_eval']), axis=1)
# else:
with open(args.preprocess_gold, "r") as f:
    data = json.load(f)
df_gold_eval = pd.read_json(data['df_gold_eval'])
df_gold_eval['gold_eval_df'] = df_gold_eval.apply(lambda x: make_dataframe_from_sparql_response(x['gold_eval']), axis=1)
df_gold_eval['gold_id_column'] = df_gold_eval.apply(lambda x: pd.DataFrame(data=x['gold_eval_df'][find_id_column(x['gold_eval_df'])]) if find_id_column(x['gold_eval_df']) != None else None, axis=1)

In [6]:
df_gold_eval.loc[df_gold_eval['gold_eval'].map(len) == 0].count()

query                     20
description               20
context                   20
prompt                    20
num_tokens                20
start_with_SELECT         20
result                    20
full_answer               20
is_skipped                20
is_prompt_too_long        20
execution                 20
executed_query            20
input                     20
target_template           20
target_raw                20
gold_eval                 20
gold_get_nested_values    20
gold_eval_df              20
gold_id_column             0
dtype: int64

In [7]:
df_merged_eval = df_eval.copy()

# Merging manually
df_merged_eval["gold_eval"] = df_merged_eval.apply(lambda x: safe_loc(x, df_gold_eval, "gold_eval", default=None), axis=1)
df_merged_eval["gold_get_nested_values"] = df_merged_eval.apply(lambda x: safe_loc(x, df_gold_eval, "gold_get_nested_values", default=None), axis=1)
df_merged_eval["gold_eval_df"] = df_merged_eval.apply(lambda x: safe_loc(x, df_gold_eval, "gold_eval_df", default=None), axis=1)
df_merged_eval["gold_id_column"] = df_merged_eval.apply(lambda x: series_or_dataframe_to_list(pd.DataFrame(data=safe_loc(x, df_gold_eval, "gold_id_column", default=None, as_serie=True))), axis=1)

In [8]:
df_merged_eval.loc[~df_merged_eval["gold_eval_df"].map(lambda x: isinstance(x, pd.DataFrame))].head()

,row,last_executed_step,to_be_executed_step,translated_prompt,status,has_error,query,description,context,prompt,...,target_template,target_raw,eval,get_nested_values,eval_df,id_column,gold_eval,gold_get_nested_values,gold_eval_df,gold_id_column
2428,"""Find all unique things and their labels, alon...",LLMTranslator,,SELECT DISTINCT ?thing ?thingLabel ?city ?city...,,False,SELECT DISTINCT ?thing ?thingLabel ?city ?city...,,"Mapping globally\nThings that are ""located in""...","<s>[INST] QUERY=""SELECT DISTINCT ?thing ?thing...",...,SELECT DISTINCT ?thing ?thingLabel ?city ?city...,SELECT DISTINCT ?thing ?thingLabel ?city ?city...,[],[],Empty DataFrame Columns: [] Index: [],None,None,None,None,[]
2430,"""Find cities that are located next to more tha...",LLMTranslator,,SELECT ?city ?cityLabel (COUNT(DISTINCT ?river...,,False,SELECT ?item ?itemLabel (COUNT(DISTINCT ?river...,Cities located next to most rivers] ([ source,,"<s>[INST] QUERY=""SELECT ?item ?itemLabel (COUN...",...,SELECT ?item ?itemLabel (COUNT(DISTINCT ?river...,SELECT ?item ?itemLabel (COUNT(DISTINCT ?river...,[],[],Empty DataFrame Columns: [] Index: [],None,None,None,None,[]
2431,"""Find the number of female employees at variou...",LLMTranslator,,SELECT ?orgLabel (COUNT(?person) AS ?count) WH...,,False,SELECT ?thing ?thingLabel ?count with {\nSELEC...,Number of female employees at different organi...,,"<s>[INST] QUERY=""SELECT ?thing ?thingLabel ?co...",...,SELECT ?thing ?thingLabel ?count with {\nSELEC...,SELECT ?thing ?thingLabel ?count with {\nSELEC...,[],[],Empty DataFrame Columns: [] Index: [],None,None,None,None,[]
2452,"""Find the most recent common ancestors between...",LLMTranslator,,SELECT ?ancestor ?ancestorLabel ?ancestorDOB ?...,,False,SELECT ?anc ?dob ?dod ?ancLabel WITH {SELECT ?...,Most recent common ancestors of George W. Bush...,,"<s>[INST] QUERY=""SELECT ?anc ?dob ?dod ?ancLab...",...,SELECT ?anc ?dob ?dod ?ancLabel WITH {SELECT ?...,SELECT ?anc ?dob ?dod ?ancLabel WITH {SELECT ?...,[],[],Empty DataFrame Columns: [] Index: [],None,None,None,None,[]
2453,"""Find authors, writers, and poets with multipl...",LLMTranslator,,SELECT ?author ?authorLabel ?authorDescription...,,False,SELECT DISTINCT ?writer ?place ?linkcount\nWHE...,,"Culture \n Literature \n Authors, writers ...","<s>[INST] QUERY=""SELECT DISTINCT ?writer ?plac...",...,SELECT DISTINCT ?writer ?place ?linkcount\nWHE...,SELECT DISTINCT ?writer ?place ?linkcount\nWHE...,[],[],Empty DataFrame Columns: [] Index: [],None,None,None,None,[]


In [9]:
df_merged_eval.columns

Index(['row', 'last_executed_step', 'to_be_executed_step', 'translated_prompt',
       'status', 'has_error', 'query', 'description', 'context', 'prompt',
       'num_tokens', 'start_with_SELECT', 'result', 'full_answer',
       'is_skipped', 'is_prompt_too_long', 'execution', 'executed_query',
       'input', 'target_template', 'target_raw', 'eval', 'get_nested_values',
       'eval_df', 'id_column', 'gold_eval', 'gold_get_nested_values',
       'gold_eval_df', 'gold_id_column'],
      dtype='object')

In [10]:
print(df_merged_eval['query'].iloc[1])

select ?building ?buildingLabel ?coord ?image WHERE
{
?building wdt:P84 wd:Q134165 ;
wdt:P625 ?coord .
OPTIONAL {?building wdt:P18 ?image}
SERVICE wikibase:label { bd:serviceParam wikibase:language "en, fr, de, pl". }
}


In [11]:
print(df_merged_eval['gold_id_column'].iloc[1])

['Point(-43.50519 -20.38465)', 'Point(-43.505111111 -20.384166666)', 'Point(-66.8728497 10.4805353)', 'Point(-43.978992 -19.848365)', 'Point(0.10611 49.49056)', 'Point(-43.9805245 -19.8549112)', 'Point(-35.22773 -5.8513)', 'Point(-43.251111111 -22.992222222)', 'Point(-51.232119 -30.045933)', 'Point(-46.654555555 -23.588416666)']


In [12]:
df_merged_eval['gold_eval_df'].iloc[1]

,building,coord,image,buildingLabel
0,http://www.wikidata.org/entity/Q18481218,Point(-43.50519 -20.38465),http://commons.wikimedia.org/wiki/Special:File...,Grande Hotel of Ouro Preto
1,http://www.wikidata.org/entity/Q18481218,Point(-43.505111111 -20.384166666),http://commons.wikimedia.org/wiki/Special:File...,Grande Hotel of Ouro Preto
2,http://www.wikidata.org/entity/Q26308703,Point(-66.8728497 10.4805353),http://commons.wikimedia.org/wiki/Special:File...,Museum of Modern Art in Caracas
3,http://www.wikidata.org/entity/Q29470059,Point(-43.978992 -19.848365),http://commons.wikimedia.org/wiki/Special:File...,Yacht Tennis Club
4,http://www.wikidata.org/entity/Q34754973,Point(0.10611 49.49056),http://commons.wikimedia.org/wiki/Special:File...,bibliothèque Oscar-Niemeyer
5,http://www.wikidata.org/entity/Q40357761,Point(-43.9805245 -19.8549112),http://commons.wikimedia.org/wiki/Special:File...,Museu Casa Kubitschek
6,http://www.wikidata.org/entity/Q45912992,Point(-35.22773 -5.8513),None,Torre de Natal
7,http://www.wikidata.org/entity/Q46011103,Point(-43.251111111 -22.992222222),http://commons.wikimedia.org/wiki/Special:File...,Rocinha footbridge
8,http://www.wikidata.org/entity/Q48878372,Point(-51.232119 -30.045933),http://commons.wikimedia.org/wiki/Special:File...,Memorial Luiz Carlos Prestes
9,http://www.wikidata.org/entity/Q50823385,Point(-46.654555555 -23.588416666),http://commons.wikimedia.org/wiki/Special:File...,Pavilhão da Bienal


In [13]:
# Computing metrics
df_merged_eval["precision"] = df_merged_eval.apply(lambda x: compute_precision(x['get_nested_values'], x['gold_get_nested_values']), axis=1)
df_merged_eval["recall"] = df_merged_eval.apply(lambda x: compute_recall(x['get_nested_values'], x['gold_get_nested_values']), axis=1)
df_merged_eval["average_precision"] = df_merged_eval.apply(lambda x: average_precision(x['get_nested_values'], x['gold_get_nested_values'], k_max=100000), axis=1)

df_merged_eval["id_precision"] = df_merged_eval.apply(lambda x: compute_precision(series_or_dataframe_to_list(x['id_column']), series_or_dataframe_to_list(x['gold_id_column'])), axis=1)
df_merged_eval["id_recall"] = df_merged_eval.apply(lambda x: compute_recall(series_or_dataframe_to_list(x['id_column']), series_or_dataframe_to_list(x['gold_id_column'])), axis=1)
df_merged_eval["id_average_precision"] = df_merged_eval.apply(lambda x: average_precision(series_or_dataframe_to_list(x['id_column']), series_or_dataframe_to_list(x['gold_id_column']), k_max=100000), axis=1)

df_merged_eval["cross_precision"] = df_merged_eval.apply(lambda x: cross_product_func(compute_precision, x['eval_df'], x['gold_eval_df'], maximization=True), axis=1)
df_merged_eval["cross_recall"] = df_merged_eval.apply(lambda x: cross_product_func(compute_recall, x['eval_df'], x['gold_eval_df'], maximization=True), axis=1)
df_merged_eval["cross_average_precision"] = df_merged_eval.apply(lambda x: cross_product_func(average_precision, x['eval_df'], x['gold_eval_df'], maximization=True, k_max=100000), axis=1)

In [14]:
m_precision = df_merged_eval['precision'].mean()
m_recall = df_merged_eval['recall'].mean()
m_fscore = 2*m_precision*m_recall/(m_precision+m_recall)

id_precision = df_merged_eval['id_precision'].mean()
id_recall = df_merged_eval['id_recall'].mean()
id_fscore = 2*id_precision*id_recall/(id_precision+id_recall)

cross_precision = df_merged_eval['cross_precision'].mean()
cross_recall = df_merged_eval['cross_recall'].mean()
cross_fscore = 2*cross_precision*cross_recall/(cross_precision+cross_recall)

mean_average_precision = df_merged_eval['average_precision'].mean()
mean_id_average_precision = df_merged_eval['id_average_precision'].mean()
mean_cross_average_precision = df_merged_eval['cross_average_precision'].mean()

In [15]:
bleu_score = corpus_bleu([[x.split()] for x in df_no_gen_fail['target_template']], [x.split() for x in df_no_gen_fail['translated_prompt']])
meteor_score = corpus_meteor(df_no_gen_fail['target_template'], df_no_gen_fail['translated_prompt'])
correct_syntax = sum(list(map(lambda y: int(y[1]), df_no_gen_fail.apply(lambda x: is_correct_SPARQL_query(x['translated_prompt']), axis=1).items()))) / len(df_no_gen_fail)

In [16]:
serie = pd.Series(data=
    {
        "model_name": args.model,
        "num_rows": len(df),
        "num_gen_fail": len(df.loc[df['has_error'] == True]),
        "num_exec_timeout": len(df_exec_timeout),
        "num_exec_fail": len(df_exec_fail),
        "num_exec_empty": len(df_exec_empty),
        "num_exec_to_eval": len(df_exec_to_eval),
        "num_eval": len(df_eval),
        "num_eval_empty": len(df_eval.loc[df_eval['eval'].map(len) == 0]),
        "bleu_score": bleu_score,
        "meteor_score": meteor_score,
        "precision": m_precision,
        "recall": m_recall,
        "f1score": m_fscore,
        "mean_average_precision": mean_average_precision,
        "id_precision": id_precision,
        "id_recall": id_recall,
        "id_f1score": id_fscore,
        "mean_id_average_precision": mean_id_average_precision,
        "cross_precision": cross_precision,
        "cross_recall": cross_recall,
        "cross_f1score": cross_fscore,
        "mean_cross_average_precision": mean_cross_average_precision,
        "correct_syntax": correct_syntax,
    })

In [17]:
serie

model_name                      Mistral-7B-Instruct-v0.2
num_rows                                             426
num_gen_fail                                          47
num_exec_timeout                                      23
num_exec_fail                                         67
num_exec_empty                                         0
num_exec_to_eval                                     336
num_eval                                             336
num_eval_empty                                       318
bleu_score                                      0.010765
meteor_score                                    0.152774
precision                                       0.002878
recall                                           0.00127
f1score                                         0.001763
mean_average_precision                          0.059855
id_precision                                    0.002978
id_recall                                       0.001244
id_f1score                     

In [18]:
df_merged_eval[['eval_df', 'gold_eval_df']].loc[df_merged_eval['precision'] > df_merged_eval['cross_precision']].head()

,eval_df,gold_eval_df
